<div class='center'>
<img src='images/seattlehouses.jpg' style='width:800px;'/>
</div>

---

# Doug and the Data Diggers

Authors: Jakub Rybicki, Chris O'Malley, Doug Mill

---

## Overview

Our task is to build an inferential linear regression model. Our model will help our stakeholder understand King County home valuations better. We will follow the assumptions of linear regression which are linearity, independence, normality, and homoscedasticity. We will also strive to have a high R^2 value, signaling that our parameters are explaining much of the total variance in house sales.

## Business Understanding

Our stakeholder is an out-of-town real estate agency interested in opening an office in King County. The county seat is Seattle.

We are acting as a consultant for our stakeholder to determine key factors in evaluating a home in King County. We want to provide our client with the math and reasoning behind local home valuations. We will discuss the important features that affect a valuation based on analysis of previous home sales in the area. We will take a look at factors such as home size & space, school district in which the home is located, upgrades & amenities, and local market conditions.

Our stakeholder will be able to evaluate the Seattle and King County real estate market by understanding the key variables that affect price from our analysis.

## Data Understanding

Our original data set includes info about King County homes that sold between May 2nd, 2014 to May 24th, 2015. The target variable is price. We removed properties that were outside of three standard deviations of the mean price. This left us with homes with a price under $1.65M. We also removed typos. That took us from 21597 entries to 21280. We then incorporated 2015 School GIS data from kingcounty.gov in order to create our 'District' variable. The District variable includes the school district that the home is located in. There are 18 school districts in King County. We cleaned the original data set and merged it with the outside data in order to create our cleaned dataset, known as 'KingSchool.csv'. It can be found in the data folder.

### *Loading our data*

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from scipy import stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.power import TTestIndPower, TTestPower
import matplotlib.pyplot as plt
%matplotlib inline
import math
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate, ShuffleSplit
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error, mean_squared_error, median_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.dummy import DummyRegressor
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
from statsmodels.stats.outliers_influence import variance_inflation_factor

## Data Preparation

For data preparation, we formatted many columns. We later decided we didn't need to use many of them. We ended up eliminating samples where the price was outside of 3 standard deviations of the mean price. We then found a typo and removed it. This is how we cleaned the data from the dataset that was included which was kc_house_data.csv. 

We incorporated 2015 School GIS data from the kingcounty.gov website. We then merged it with kc_house_data.csv on 4 different keys; 'id', 'lat', 'long', and 'zipcode'. This plotted the entries against district lines, and formed a column in which each entry was classified by the district that it was located in. There are 18 school districts in King County; Mercer Island, Bellevue, Seattle, Lake Washington, Vashon Island, Issaquah, Shoreline, Northshore, Snoqualmie Valley, Riverview, Highline, Renton, Skykomish, Enumclaw, Tahoma, Tukwila, Kent, and Federal Way.

Since district contained 18 categories, we changed it into dummy variables. By using get_dummies, District was transformed into 18 seperate variables. Each variable contains a 0 or 1. 0 signals that the house was not located in that district while 1 signals that it is present in that district.

We picked which columns that we wanted to focus on originally from a heatmap of correlations. We ran train test splits for every single model that we did. We also made sure to run StandardScaler on every single model we did in order to scale them.

In [2]:
df  = pd.read_csv('data/KingSchool.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'data/KingSchool.csv'

In [ ]:
categoricals = ['District']
dummies = pd.get_dummies(df[categoricals], prefix=categoricals, drop_first=True)
df = df.drop(categoricals, axis=1)
df = pd.concat([df, dummies], axis=1)

In [ ]:
df.drop(['date'],axis=1,inplace=True)
df = df.astype(np.float64)
df['District_FEDERAL_WAY'] = df['District_FEDERAL WAY']
df['District_LAKE_WASHINGTON'] = df['District_LAKE WASHINGTON']
df['District_MERCER_ISLAND'] = df['District_MERCER ISLAND']
df['District_SNOQUALMIE_VALLEY'] = df['District_SNOQUALMIE VALLEY']
df['District_VASHON_ISLAND'] = df['District_VASHON ISLAND']
df.drop(['District_FEDERAL WAY','District_LAKE WASHINGTON','District_MERCER ISLAND',
                         'District_SNOQUALMIE VALLEY','District_VASHON ISLAND'],axis=1,inplace=True)

In [ ]:
df_dummies = ['District_BELLEVUE','District_ENUMCLAW','District_FEDERAL_WAY','District_HIGHLINE',
                         'District_ISSAQUAH','District_KENT','District_LAKE_WASHINGTON','District_MERCER_ISLAND',
                         'District_NORTHSHORE','District_RENTON','District_RIVERVIEW','District_SEATTLE',
                         'District_SHORELINE','District_SKYKOMISH','District_SNOQUALMIE_VALLEY','District_TAHOMA',
                         'District_TUKWILA','District_VASHON_ISLAND']

An already cleaned kc_house_data.csv was used to extract school data and map using ARCGIS rendering the following formatting obsolete, however it will still be listed for clarity.

### *Column formatting & dealing with missing values*

In [ ]:
# Converting Grade to Numeric
# df['grade'] = df['grade'].str.slice(0,2).str.strip()
# df['grade'] = df['grade'].astype(np.int64)

In [ ]:
# Converting View to Numeric
# df['view'].fillna('NONE', inplace=True)
# df['view'].replace('NONE', '0', inplace=True)
# df['view'].replace('FAIR', '1', inplace=True)
# df['view'].replace('AVERAGE', '2', inplace=True)
# df['view'].replace('GOOD', '3', inplace=True)
# df['view'].replace('EXCELLENT', '4', inplace=True)
# df['view'] = df['view'].astype(np.int64)

### *Dropping outliers*

In [ ]:
#Dropping outliers
# price_mean = np.mean(df['price'])
# cut_off = np.std(df['price']) * 3
# df.drop(df[df['price'] > price_mean + cut_off].index, inplace = True)
# df.drop(df[df['price'] < price_mean - cut_off].index, inplace = True)

### *Presentation Charting*

In [ ]:
fig, ax = plt.subplots(figsize=(20,15))
sns.histplot(df['price'], ax=ax, bins=50, kde=True, linewidth=2)
plt.plot([df['mean_price'], df['mean_price']], [0, 1500], linewidth=5, color='red')
plt.title('Average Price Distribution with Mean Line', fontsize=30, fontweight='bold')
plt.xlabel('Sale Price in Millions', fontsize=25, fontweight='bold')
plt.xticks(fontsize=20)
plt.ylabel('Amount Sold', fontsize=25, fontweight='bold')
plt.yticks(fontsize=20);

In [ ]:
fig, ax = plt.subplots(figsize=(12, 9))
df_corrs = df.corr().abs()['price'].sort_values(ascending=False)
df_corrs = pd.Series(df_corrs[['grade', 'sqft_living', 'view', 'yr_built']])
ax.bar(df_corrs.index, df_corrs.values)
plt.ylim(0, 1.0)
plt.title('Percent Correlation with Price', fontsize=30, fontweight='bold')
plt.xlabel('Features', fontsize=25, fontweight='bold')
plt.xticks( fontsize=20)
plt.ylabel('Percent Correlation', fontsize=25, fontweight='bold')
plt.yticks(fontsize=20)
ax.set_yticklabels(['0%', '20%', '40%', '60%', '80%', '100%']);

In [ ]:
df_no_district = df[['price', 'sqft_living', 'view', 'grade', 'yr_built']]
fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(16,12))

columns = df_no_district.columns

for col, ax in zip(columns, axes.flatten()):
    ax.hist(df_no_district[col].dropna(), bins='auto')
    ax.set_title(col)
    
fig.tight_layout()

## Modeling

We created 6 models. Baseline model, First model, Good model, Great model, Awesome model, and Amazing model.
In the end, we decided to go with Great model as our Final model. 

Our Final model, known as "Great Model" has the follow predictors:
1. Square feet living
2. School Districts
3. Building Grade

We decided Great Model is adequate with an R-squared value of .72 with only 3 predictors. The simplicity of it makes it easy to understand and use while also explaining such a high amount of the variance in price.

In [ ]:
corr = df.iloc[:, 2:21].corr()
# Set up figure and axes
fig, ax = plt.subplots(figsize=(12, 15))
# Plot a heatmap of the correlation matrix, with both
# numbers and colors indicating the correlations
sns.heatmap(
    # Specifies the data to be plotted
    data=corr,
    # The mask means we only show half the values,
    # instead of showing duplicates. It's optional.
    mask=np.triu(np.ones_like(corr, dtype=bool)),
    # Specifies that we should use the existing axes
    ax=ax,
    # Specifies that we want labels, not just colors
    annot=True,
    # Customizes colorbar appearance
    cbar_kws={"label": "Correlation", "orientation": "horizontal", "pad": .1, "extend": "both"},
    annot_kws={"fontsize":10}
)
# Customize the plot appearance
ax.set_title("Heatmap of Correlation Between Attributes (Including Target)");

### *Helper Functions*

In [ ]:
def linearity_plot(x, y):
    x = pd.DataFrame(x)
    y = pd.DataFrame(y)
    cols = 1
    if x.shape[1] % 2 == 0:
        cols = 2
    elif x.shape[1] < 2:
        cols = 1
    else:
        cols = 3
    rows = 1
    if x.shape[1] // cols == 0:
        rows = 1
    elif x.shape[1] % cols > 0:
        rows = x.shape[1] // cols + 1
    else:
        rows = rows = x.shape[1] // cols 
    fig, axes = plt.subplots(ncols=cols, nrows=rows, figsize=(16,10), squeeze=False)
    fig.set_tight_layout(True)
    for index, col in enumerate(x.columns):
        ax = axes[index//cols][index%cols]
        ax.scatter(x[col], y, alpha=0.2)
        ax.set_xlabel(col, fontsize=20)
        ax.set_ylabel("Listing price", fontsize=20)

In [ ]:
def train_lr_randomly(data, sample_pt=None, ntimes=500):
    '''
    Takes in features & targets from `data` to train a linear regression with a
    random sample `ntimes`. It then returns a list of R2 scores, RMSEs, and the 
    predictions from a provided data point of features `sample_pt`.
    '''
    # To save all of our predictions
    r2 = []
    rmse = []
    # Only return predictions if there is something to predict (sample_pt given)
    point_preds = [] if (sample_pt is not None) else None

    # We'll repeat this little experiment to see how the model does
    for i in range(ntimes):
        # Creating a random sample of data to train on
        df_sample = data.sample(5000, replace=True)
        y = df_sample.price
        X = df_sample.drop('price', axis=1)

        # Our linear regression model about to be trained
        lr = LinearRegression()
        lr.fit(X, y)

        # Making predictions & evaluating on the data we used to train the model
        y_hat = lr.predict(X)
        rmse.append(np.sqrt(mean_squared_error(y, y_hat)))
        r2.append(lr.score(X, y))

        # Making a prediction on the one point the model definitely never saw
        if sample_pt is not None:
            y_hat_pt = lr.predict(sample_pt)
            # Getting just the single point to add into list
            point_preds.append(y_hat_pt[0])
    
    return r2, rmse, point_preds

In [ ]:
def test_box(formula = df.columns[1:6]):
    sum_df = df[formula]
    sum_sample = sum_df.sample(1)
    sum_sample_price = sum_sample.iloc[0]['price']
    sum_sample_pt = sum_sample.drop('price', axis=1)
    print(sum_sample_price)
    # Show my random sample off
    print(f'Price of sample: ${sum_sample_price}')    
    
    # Run 100 linear regression trainings on some random data from df and compare
    # it with the random sample point
    r2_sum, rmse_sum, pt_preds_sum = train_lr_randomly(
                                                            data=sum_df, 
                                                            sample_pt=sum_sample_pt,
                                                            ntimes=500                            
    )
    
    ax = sns.boxplot(x=pt_preds_sum);
    ax = sns.swarmplot(x=pt_preds_sum, color='orange', ax=ax)
    ax.set_title(f'Predicting Sample Pt Price: ${sum_sample_price:,.2f}');

### Baseline Model

Our baseline model takes the mean price of the whole dataset. It is a horizontal line which sits at the mean price.

In [ ]:
baseline_y = df['price']
baseline_X = df.drop(['price'], axis=1)

In [ ]:
baseline_X_train, baseline_X_test, baseline_y_train, baseline_y_test = train_test_split(baseline_X, baseline_y, random_state=42)

In [ ]:
scaler = StandardScaler()
baseline_X_train_scaled = scaler.fit_transform(baseline_X_train)
baseline_X_test_scaled = scaler.fit_transform(baseline_X_test)



In [ ]:
print(f'Train Mean: {baseline_y_train.mean()}, Test Mean: {baseline_y_test.mean()}')

In [ ]:
dummy_regr = DummyRegressor(strategy="mean")
dummy_regr.fit(baseline_X_train_scaled, baseline_y_train)
dummy_regr.predict(baseline_X_train_scaled)
dummy_regr.score(baseline_X_test_scaled, baseline_y_test)

In [ ]:
baseline_X_train_scaled = pd.DataFrame(baseline_X_train_scaled)
baseline_X_train_scaled.columns = baseline_X.columns
baseline_X_test_scaled = pd.DataFrame(baseline_X_test_scaled)
baseline_X_test_scaled.columns = baseline_X.columns

### *Interpreting the baseline model*

Our baseline model definitely follows the linearity assumption. The distribution of errors is slightly skewed right. Many of the VIF values are above 10 and this indicates that there is high multicollinearity. It passes the homoscedasticity test.

### *Assumptions*

In [ ]:
#LINEARITY
df['mean_price']=df['price'].mean()
linearity_plot(df['price'], df['mean_price'])

In [ ]:
sns.lmplot(x='sqft_living', y='price', data=df, line_kws={'color': 'black'});

In [ ]:
#NORMALITY
# Normal distribution of errors check
baseline_model = LinearRegression()
baseline_model.fit(baseline_X_train_scaled, baseline_y_train)
baseline_model.score(baseline_X_test_scaled, baseline_y_test)
preds = baseline_model.predict(baseline_X_test_scaled)
residuals = (baseline_y_test - preds)
sm.graphics.qqplot(residuals, dist=stats.norm, line='r', fit=True);

In [ ]:
#Checking MULTICOLLINEARITY 
# (VIF NEEDS TO BE <5 ALL CATEGORIES)
vif = [variance_inflation_factor(baseline_X_train_scaled.values, i) for i in range(baseline_X_train_scaled.shape[1])]
pd.Series(vif, index=baseline_X_train_scaled.columns, name="Variance Inflation Factor")

In [ ]:
#HOMOSCEDASTICITY
fig, ax = plt.subplots()
ax.scatter(preds, residuals, alpha=0.5)
ax.plot(preds, [0 for i in range(len(baseline_X_test_scaled))], color='black')
ax.set_xlabel("Predicted Value")
ax.set_ylabel("Actual - Predicted Value");

### *Mass Tests*

In [ ]:
test_box()

### *Dropping Unnecessary Columns*

In [ ]:
df.drop(['id','bedrooms','bathrooms','sqft_lot','floors','waterfront','condition','sqft_above','sqft_basement', 'yr_renovated',
                         'zipcode','lat','long','sqft_living15','sqft_lot15','renovated_less_10yrs',
                         'has_basement','Distance_to_School_ft','mean_price'], axis=1,inplace=True)

### First Model: Square Feet Living & Price

Our first model includes Square Feet Living plotted against Price.

In [ ]:
first_y = df["price"]
first_formula = ['sqft_living']
first_full_formula = ['price'] + first_formula
first_X = df[first_formula]

In [ ]:
first_X_train, first_X_test, first_y_train, first_y_test = train_test_split(first_X, first_y, random_state=42)

In [ ]:
scaler = StandardScaler()
first_X_train_scaled = scaler.fit_transform(first_X_train)
first_X_test_scaled = scaler.fit_transform(first_X_test)

In [ ]:
first_model = LinearRegression()

# Fit the model on X_train_final and y_train
first_model.fit(first_X_train_scaled, first_y_train)

# Score the model on X_test_final and y_test
# (use the built-in .score method)
first_model.score(first_X_test_scaled, first_y_test)

In [ ]:
print(f'Train RMSE: {mean_squared_error(first_y_train, first_model.predict(first_X_train_scaled), squared=False)}')
print(f'Test RMSE: {mean_squared_error(first_y_test, first_model.predict(first_X_test_scaled), squared=False)}')

In [ ]:
sm.OLS(first_y_train, sm.add_constant(first_X_train)).fit().summary()

In [ ]:
first_X_train_scaled = pd.DataFrame(first_X_train_scaled)
first_X_train_scaled.columns = first_X.columns
first_X_test_scaled = pd.DataFrame(first_X_test_scaled)
first_X_test_scaled.columns = first_X.columns

In [ ]:
fig, ax = plt.subplots()
ax.scatter(first_X_train[first_formula], first_y_train, alpha=0.5, color ='white', edgecolor='blue')
ax.set_xlabel('Square Feet Living')
ax.set_ylabel("Price (1M's)")
ax.set_title('Square Feet to Price')
plt.plot(np.unique(first_X_train['sqft_living']), np.poly1d(np.polyfit(first_X_train['sqft_living'], first_y_train, 1))
         (np.unique(first_X_train['sqft_living'])),linewidth=3.0,color='black');


### *Interpreting the first model*

Square Feet Living drives up Price. It has an R-squared of .445. It explains 44.5% of the variance in price by itself. It is the most correlated variable and individually most significant in our findings. Our first model definitely follows the linearity assumption. The distribution of errors is very slightly heavy on the tails. Multicollinearity is not applicable for one predictor. It passes the homoscedasticity test.

### *Assumptions*

In [ ]:
#LINEARITY
linearity_plot(first_X_train, first_y_train)

In [ ]:
#NORMALITY
import scipy.stats as stats
preds_1st = first_model.predict(first_X_test_scaled)
residuals = (first_y_test - preds_1st)
sm.graphics.qqplot(residuals, dist=stats.norm, line='r', fit=True);

In [ ]:
#HOMOSCEDASTICITY
fig, ax = plt.subplots()
ax.scatter(preds_1st, residuals, alpha=0.5)
ax.plot(preds, [0 for i in range(len(first_X_test_scaled))], color='black')
ax.set_xlabel("Predicted Value")
ax.set_ylabel("Actual - Predicted Value");

In [ ]:
test_box(first_full_formula)

### Good Model: Square Feet Living and School District

For the good model, we have 2 predictors. Those predictors are square feet living and school district. Our school district variable consists of the 18 districts located in King County. Those districts are Bellevue, Enumclaw, Federal Way, Highline, Issaquah, Kent, Lake Washington, Mercer Island, Northshore, Renton, Riverview, Seattle, Shoreline, Skykomish, Snoqualmie Valley, Tahoma, Tukwila, Vashon Island.

In [ ]:
good_y = df["price"]
good_formula = first_formula + ['District_BELLEVUE','District_ENUMCLAW','District_FEDERAL_WAY',
                 'District_HIGHLINE','District_ISSAQUAH','District_KENT','District_LAKE_WASHINGTON','District_MERCER_ISLAND',
                 'District_NORTHSHORE','District_RENTON','District_RIVERVIEW','District_SEATTLE','District_SHORELINE',
                 'District_SKYKOMISH','District_SNOQUALMIE_VALLEY','District_TAHOMA','District_TUKWILA',
                 'District_VASHON_ISLAND']
good_full_formula = ['price'] + good_formula
good_X = df[good_formula]

In [ ]:
good_X_train, good_X_test, good_y_train, good_y_test = train_test_split(good_X, good_y, random_state=42)

In [ ]:
scaler = StandardScaler()
good_X_train_scaled = scaler.fit_transform(good_X_train)
good_X_test_scaled = scaler.fit_transform(good_X_test)



In [ ]:
good_model = LinearRegression()

# Fit the model on X_train_final and y_train
good_model.fit(good_X_train_scaled, good_y_train)

# Score the model on X_test_final and y_test
# (use the built-in .score method)
good_model.score(good_X_test_scaled, good_y_test)

In [ ]:
print(f'Train RMSE: {mean_squared_error(good_y_train, good_model.predict(good_X_train_scaled), squared=False)}')
print(f'Test RMSE: {mean_squared_error(good_y_test, good_model.predict(good_X_test_scaled), squared=False)}')

In [ ]:
sm.OLS(good_y_train, sm.add_constant(good_X_train)).fit().summary()

In [ ]:
good_X_train_scaled = pd.DataFrame(good_X_train_scaled)
good_X_train_scaled.columns = good_X.columns
good_X_test_scaled = pd.DataFrame(good_X_test_scaled)
good_X_test_scaled.columns = good_X.columns

### *Interpreting the model*

Good model has an R-squared of .683. Square feet living combined with District explains 68.3% of the variance in price. The inclusion of District raised our R-squared from .44 to .68, for an increase of .24. Our good model definitely follows the linearity assumption. Square feet living follows linearity, and district is linear by definition as a categorical variable. The distribution of errors is skewed right. Good model has low multicollinearity in general, with all VIF below 5 except for District_SEATTLE at 7. It passes the homoscedasticity test.

### *Assumptions*

In [ ]:
linearity_plot(good_X_train['sqft_living'], good_y_train)

In [ ]:
#NORMALITY
good_X_test_scaled = pd.DataFrame(good_X_test_scaled)
preds_good = good_model.predict(good_X_test_scaled)
residuals = (good_y_test - preds_good)
sm.graphics.qqplot(residuals, dist=stats.norm, line='r', fit=True);

In [ ]:
#Checking MULTICOLLINEARITY 
# (VIF NEEDS TO BE <5 ALL CATEGORIES)
vif = [variance_inflation_factor(good_X_train_scaled.values, i) for i in range(good_X_train_scaled.shape[1])]
pd.Series(vif, index=good_X_train_scaled.columns, name="Variance Inflation Factor")

In [ ]:
#HOMOSCEDASTICITY
fig, ax = plt.subplots()

ax.scatter(preds_good, residuals, alpha=0.5)
ax.plot(preds_good, [0 for i in range(len(good_X_test_scaled))], color='black')
ax.set_xlabel("Predicted Value")
ax.set_ylabel("Actual - Predicted Value");

In [ ]:
#HOMOSCEDASTICITY BORDERLINE

In [ ]:
test_box(good_full_formula)

### Great Model: Square feet living, School districts, and Grade

The Great model uses square feet living, school districts, and grade as our predictors.

In [ ]:
great_y = df["price"]
great_formula = good_formula + ['grade']
great_full_formula = ['price'] + great_formula
great_X = df[great_formula]

In [ ]:
great_X_train, great_X_test, great_y_train, great_y_test = train_test_split(great_X, great_y, random_state=42)

In [ ]:
scaler = StandardScaler()
great_X_train_scaled = scaler.fit_transform(great_X_train)
great_X_test_scaled = scaler.fit_transform(great_X_test)

In [ ]:
great_model = LinearRegression()

# Fit the model on X_train_final and y_train
great_model.fit(great_X_train_scaled, great_y_train)

# Score the model on X_test_final and y_test
# (use the built-in .score method)
great_model.score(great_X_test_scaled, great_y_test)

In [ ]:
print(f'Train RMSE: {mean_squared_error(great_y_train, great_model.predict(great_X_train_scaled), squared=False)}')
print(f'Test RMSE: {mean_squared_error(great_y_test, great_model.predict(great_X_test_scaled), squared=False)}')

In [ ]:
sm.OLS(great_y_train, sm.add_constant(great_X_train)).fit().summary()

In [ ]:
great_X_train_scaled = pd.DataFrame(great_X_train_scaled)
great_X_train_scaled.columns = great_X.columns
great_X_test_scaled = pd.DataFrame(great_X_test_scaled)
great_X_test_scaled.columns = great_X.columns

### *Interpreting the model*

Great model has an R-squared of .723. Square feet living, District, and Grade explains 72.3% of the variance in price. This is our first model which crosses the .7 R-squared threshold. The inclusion of Grade raised our R-squared from .68 to .72, for an increase of .04. Our great model definitely follows the linearity assumption, given that Districts are linear by definition, and Grade is also linear. The distribution of errors is unfortunately skewed right. Great model has low multicollinearity in general, with all VIF below 5 except for District_SEATTLE at 7. It passes the homoscedasticity test.

### *Assumptions*

In [ ]:
# Linearity
linearity_plot(great_X_train[['sqft_living', 'grade']], great_y_train)

In [ ]:
#NORMALITY
great_X_test_scaled = pd.DataFrame(great_X_test_scaled)
preds_great = great_model.predict(great_X_test_scaled)
residuals = (great_y_test - preds_great)
sm.graphics.qqplot(residuals, dist=stats.norm, line='r', fit=True);

In [ ]:
#Checking MULTICOLLINEARITY 
# (VIF NEEDS TO BE <5 ALL CATEGORIES)
vif = [variance_inflation_factor(great_X_train_scaled.values, i) for i in range(great_X_train_scaled.shape[1])]
pd.Series(vif, index=great_X_train_scaled.columns, name="Variance Inflation Factor")

In [ ]:
#HOMOSCEDASTICITY
fig, ax = plt.subplots()

ax.scatter(preds_great, residuals, alpha=0.5)
ax.plot(preds_great, [0 for i in range(len(great_X_test_scaled))], color='black')
ax.set_xlabel("Predicted Value")
ax.set_ylabel("Actual - Predicted Value");

In [ ]:
test_box(great_full_formula)

### Awesome Model: sqft_living, school districts, grade, year built

Awesome model includes square feet living, school districts, grade, and year built as predictors.

In [ ]:
awesome_y = df["price"]
awesome_formula = great_formula + ['yr_built']
awesome_full_formula = ['price'] + awesome_formula
awesome_X = df[awesome_formula]

In [ ]:
awesome_X_train, awesome_X_test, awesome_y_train, awesome_y_test = train_test_split(awesome_X, awesome_y, random_state=42)

In [ ]:
scaler = StandardScaler()
awesome_X_train_scaled = scaler.fit_transform(awesome_X_train)
awesome_X_test_scaled = scaler.fit_transform(awesome_X_test)

In [ ]:
awesome_model = LinearRegression()

# Fit the model on X_train_final and y_train
awesome_model.fit(awesome_X_train_scaled, awesome_y_train)

# Score the model on X_test_final and y_test
# (use the built-in .score method)
awesome_model.score(awesome_X_test_scaled, awesome_y_test)

In [ ]:
print(f'Train RMSE: {mean_squared_error(awesome_y_train, awesome_model.predict(awesome_X_train_scaled), squared=False)}')
print(f'Test RMSE: {mean_squared_error(awesome_y_test, awesome_model.predict(awesome_X_test_scaled), squared=False)}')

In [ ]:
sm.OLS(awesome_y_train, sm.add_constant(awesome_X_train)).fit().summary()

In [ ]:
awesome_X_train_scaled = pd.DataFrame(awesome_X_train_scaled)
awesome_X_train_scaled.columns = awesome_X.columns
awesome_X_test_scaled = pd.DataFrame(awesome_X_test_scaled)
awesome_X_test_scaled.columns = awesome_X.columns

### *Interpreting the model*

Awesome model has an R-squared of .742. Building on Great model, adding yr_built explains 74.2% of the variance in price. The inclusion of yr_built raised our R-squared from .72 to .74, for an increase of .02. Our awesome model mostly follow the linearity assumption, but the linearity of yr_built is vague. The distribution of errors is skewed right. Good model has low multicollinearity in general, with all VIF below 5 except for District_SEATTLE at 7. It passes the homoscedasticity test.

### *Assumptions*

In [ ]:
# Linearity
linearity_plot(awesome_X_train[['sqft_living', 'grade', 'yr_built']], awesome_y_train)


In [ ]:
#NORMALITY
awesome_X_test_scaled = pd.DataFrame(awesome_X_test_scaled)
preds_awesome = awesome_model.predict(awesome_X_test_scaled)
residuals = (awesome_y_test - preds_awesome)
sm.graphics.qqplot(residuals, dist=stats.norm, line='r', fit=True);

In [ ]:
#Checking MULTICOLLINEARITY 
# (VIF NEEDS TO BE <5 ALL CATEGORIES)
vif = [variance_inflation_factor(awesome_X_train_scaled.values, i) for i in range(awesome_X_train_scaled.shape[1])]
pd.Series(vif, index=awesome_X_train_scaled.columns, name="Variance Inflation Factor")

In [ ]:
#HOMOSCEDASTICITY
fig, ax = plt.subplots()

ax.scatter(preds_awesome, residuals, alpha=0.5)
ax.plot(preds_awesome, [0 for i in range(len(awesome_X_test_scaled))], color='black')
ax.set_xlabel("Predicted Value")
ax.set_ylabel("Actual - Predicted Value");

In [ ]:
test_box(awesome_full_formula)

### Amazing Model: sqft_living, school districts, grade, year built, view

Amazing model includes square feet living, school districts, grade, year built, and view as predictors.
This makes for a total of 5 predictors.

In [ ]:
amazing_y = df["price"]
amazing_formula = awesome_formula + ['view']
amazing_full_formula = ['price'] + amazing_formula
amazing_X = df[amazing_formula]

In [ ]:
amazing_X_train, amazing_X_test, amazing_y_train, amazing_y_test = train_test_split(amazing_X, amazing_y, random_state=42)

In [ ]:
scaler = StandardScaler()
amazing_X_train_scaled = scaler.fit_transform(amazing_X_train)
amazing_X_test_scaled = scaler.fit_transform(amazing_X_test)

In [ ]:
amazing_model = LinearRegression()

# Fit the model on X_train_final and y_train
amazing_model.fit(amazing_X_train_scaled, amazing_y_train)

# Score the model on X_test_final and y_test
# (use the built-in .score method)
amazing_model.score(amazing_X_test_scaled, amazing_y_test)

In [ ]:
print(f'Train RMSE: {mean_squared_error(amazing_y_train, amazing_model.predict(amazing_X_train_scaled), squared=False)}')
print(f'Test RMSE: {mean_squared_error(amazing_y_test, amazing_model.predict(amazing_X_test_scaled), squared=False)}')

In [ ]:
sm.OLS(amazing_y_train, sm.add_constant(amazing_X_train)).fit().summary()

In [ ]:
amazing_X_train_scaled = pd.DataFrame(amazing_X_train_scaled)
amazing_X_train_scaled.columns = amazing_X.columns
amazing_X_test_scaled = pd.DataFrame(amazing_X_test_scaled)
amazing_X_test_scaled.columns = amazing_X.columns

### Touching on School Districts

Since school districts is a more complex variable, we wanted to take some time to explain it. It was originally one column with 18 categores. We created dummy variables in order to model it. The result of that is 18 columns. Here, we have modeled all the districts by themselves in an OLS. We have taken data from the amazing train set for this OLS.

In [ ]:
temp_df = amazing_X_train.copy().iloc[:,1:-3]
temp_df
sm.OLS(amazing_y_train, sm.add_constant(temp_df)).fit().summary()

### *Interpreting the model*

Amazing model has an R-squared of .756. This is the highest R-squared value we were able to reach. 
This was built by using square feet living, school district, grade, year built, and view.
The inclusion of view raised our R-squared from .742 to .756, for an increase of .14.
Our amazing model has a couple predictors that are sketchy with the linearity assumption. Those variables are view and yr_built. This is part of the reason we decided not to use amazing model or awesome model as our final model. The distribution of errors is skewed right. Amazing model has low multicollinearity in general, with all VIF below 5 except for District_SEATTLE at 7. It passes the homoscedasticity test.

## Assumptions

In [ ]:
# Linearity
linearity_plot(amazing_X_train[['sqft_living', 'grade', 'yr_built', 'view']], amazing_y_train)

In [ ]:
#NORMALITY
amazing_X_test_scaled = pd.DataFrame(amazing_X_test_scaled)
preds_amazing = amazing_model.predict(amazing_X_test_scaled)
residuals = (amazing_y_test - preds_amazing)
sm.graphics.qqplot(residuals, dist=stats.norm, line='r', fit=True);

In [ ]:
#Checking MULTICOLLINEARITY 
# (VIF NEEDS TO BE <5 ALL CATEGORIES)
vif = [variance_inflation_factor(amazing_X_train_scaled.values, i) for i in range(amazing_X_train_scaled.shape[1])]
pd.Series(vif, index=amazing_X_train_scaled.columns, name="Variance Inflation Factor")

In [ ]:
#HOMOSCEDASTICITY
fig, ax = plt.subplots()

ax.scatter(preds_amazing, residuals, alpha=0.5)
ax.plot(preds_amazing, [0 for i in range(len(amazing_X_test_scaled))], color='black')
ax.set_xlabel("Predicted Value")
ax.set_ylabel("Actual - Predicted Value");

In [ ]:
test_box(amazing_full_formula)

# Regression Results - Great Model

We decided to use Great Model as our Final model. This model did very well with 3 predictors. Those predictors are square feet living, school district, and grade. The reasoning for selecting Great Model even though some of the following models had higher R-squared values was because it was simple, yet understandable. The following models seemed to muddy the waters when it came to the linearity assumption. 

The Great Model is able to explain 72.3% of the variance in price of King County Home Sales in 2014-15 with just 3 predictors.
This is a good R-squared value as it is above the 70% threshold. The model had a Train RMSE of 137070.49516065692 and a Test RMSE of 136447.31048874996. These are very close to each other, meaning the fit of the model is not underfit or overfit.

The Great Model does a great job at helping our stakeholder, the real estate agency, understand the gist of what drives home prices in the King County area. More specifically, it gives invaluable information about which school district they should target for either top end or bottom end location values.

In [ ]:
sm.OLS(great_y_train, sm.add_constant(great_X_train)).fit().summary()

In [ ]:
print(f'Train RMSE: {mean_squared_error(great_y_train, great_model.predict(great_X_train_scaled), squared=False)}')
print(f'Test RMSE: {mean_squared_error(great_y_test, great_model.predict(great_X_test_scaled), squared=False)}')

# Conclusion

Our model showed us that the most important predictors to look at in a home are going to be square feet living, school district, and building grade. To relate this back to our stakeholder, we want to inform the real estate agency to focus on these elements in their deals. We can also help them by allowing them to plot different values in our model and giving them a rough valuation based on these variables.

Our model separates each home into 3 basic elements that contribute to value:
1. Home size & usable space (sqft_living)
2. Location (school district)
3. Upgrades, amenities, & design elements (grade)

To improve our model in the future, we could bin house sales by neighborhood. We could then compare home sales within individual neighborhood and model what features accounted for the discrepancies in sales price. This would give us a more detailed look at what features distinguish home valuations. This would also help the stakeholder with their offers and listings in a more detailed and refined level. More data would also be very useful. Another thing we could improve on is using more location metrics. The location metrics we would want to add include proximity to shopping, amenities, entertainment, recreation, airports, parks, schools, landmarks, tourist attractions, and employment opportunities. 